# Mood prediction with TensorFlow
Trying to predict the mood of a user each day with:
- Mood rated between 0 and 5
- Weather rated between 0 and 5
- Temperature
- Month of the year
- Day of the week between 0 and 6

In [1]:
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input

In [2]:
mood_history = pd.read_csv('mood-history.csv').sort_values('timestamp')
mood_history.head()

,timestamp,mood,weather,temperature,month,day,weekday
0,1616320024,5,5,8,3,21,6
1,1616406424,3,4,8,3,22,0
2,1616579224,4,5,9,3,23,1
3,1616579224,3,5,9,3,24,2
4,1616665624,2,4,7,3,25,3


## Data normalization

Normalizes some input features to make them all between 0 and 1.

Applies `(x - min) / (max - min)` for all the features.

In [3]:
mood_history['mood'] = mood_history['mood'].apply(lambda x: x / 5)
mood_history['weather'] = mood_history['weather'].apply(lambda x: x / 5)
mood_history['temperature'] = mood_history['temperature'].apply(lambda x: (x + 30) / (50 + 30))

mood_history

,timestamp,mood,weather,temperature,month,day,weekday
0,1616320024,1.0,1.0,0.4750,3,21,6
1,1616406424,0.6,0.8,0.4750,3,22,0
2,1616579224,0.8,1.0,0.4875,3,23,1
3,1616579224,0.6,1.0,0.4875,3,24,2
4,1616665624,0.4,0.8,0.4625,3,25,3
5,1616752024,0.4,0.6,0.4375,3,26,4
6,1616838424,1.0,1.0,0.5250,3,27,5


In [4]:
X_train = mood_history[[
            'weather',
            'temperature',
            'month',
            'day',
            'weekday'
]]
X_train.columns = range(X_train.shape[1])

X_train.head()

,0,1,2,3,4
0,1.0,0.4750,3,21,6
1,0.8,0.4750,3,22,0
2,1.0,0.4875,3,23,1
3,1.0,0.4875,3,24,2
4,0.8,0.4625,3,25,3


In [5]:
Y_train = mood_history[[
            'mood'
]]
Y_train.columns = range(Y_train.shape[1])

Y_train.head()

,0
0,1.0
1,0.6
2,0.8
3,0.6
4,0.4


In [6]:
model = Sequential()

model.add(Input(shape=(5,)))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))

In [7]:
model.compile(optimizer='sgd', loss='mean_squared_error')

In [8]:
model.fit(X_train, Y_train, batch_size=1, epochs=10)

Epoch 1/10
7/7 [==============================] - 0s 1ms/step - loss: 0.9890
Epoch 2/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0980
Epoch 3/10
7/7 [==============================] - 0s 1ms/step - loss: 0.1022
Epoch 4/10
7/7 [==============================] - 0s 1ms/step - loss: 0.1171
Epoch 5/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0884
Epoch 6/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0763
Epoch 7/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0928
Epoch 8/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0965
Epoch 9/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0472
Epoch 10/10
7/7 [==============================] - 0s 1ms/step - loss: 0.1128


In [9]:
predictions = model.predict(pd.DataFrame([[
            1,
            1,
            3,
            28,
            6
]]))

predictions

array([[0.94317734]], dtype=float32)